
# Objective
This notebook demonstrates how to setup Elasticsearch as a vector database (VectorDB) to support LLM functions that can provide an intelligent query layer.


- **Elasticsearch as the VectorDB**: Acts as the core search engine, storing and retrieving dense vector embeddings efficiently.
- **Search Templates**: Marry index capabilities to query parameters, enabling dynamic query generation and structured search execution.

This combination enables a more sophisticated search experience, leveraging both structured and unstructured data retrieval methods.


## Elasticsearch Setup

You will need an Elasticsearch instance that has an **enterprise** plan entitlement to complete this walk thru. This walk thru was built and tested on Elastic Serverless and Elastic Cloud Hosted 9.0.0 on Azure.

- Details on how to create a new Elastic serverless project can be found [here](https://www.elastic.co/docs/solutions/search/serverless-elasticsearch-get-started#elasticsearch-get-started-create-project)

- Details on how to create an API key can be found [here](https://www.elastic.co/docs/solutions/search/search-connection-details#create-an-api-key-cloud-self-managed)


## Install libraries

In [1]:
%pip install elasticsearch openai streamlit python-dotenv tqdm ipywidgets

Note: you may need to restart the kernel to use updated packages.


# Define imports and load environment variables

In [2]:
import os
import json
from elasticsearch import Elasticsearch, helpers, TransportError, ConnectionError
import requests
import requests, json
import time
from dotenv import load_dotenv
from tqdm.notebook import tqdm

load_dotenv()

True

## Environment variables
Supply elasticsearch serverless cloud ID & API Key

### Elastic 
```
ELASTIC_URL
ELASTIC_API_KEY
```

### Azure
```
AZURE_MAPS_API_KEY
GEOCODE_URL

AZURE_OPENAI_API_KEY
AZURE_OPENAI_DEPLOYMENT_NAME
AZURE_OPENAI_API_VERSION
AZURE_OPENAI_ENDPOINT
```

**Details on each secret is defined in** [part 1](https://github.com/elastic/elasticsearch-labs/blob/main/supporting-blog-content/unifying-elastic-vector-database-and-llms-for-intelligent-query/Unifying_Elastic_Vector_Database_and_LLMs_for_Intelligent_Query.ipynb)

## Set local variables and test elasticsearch connection

In [3]:
# Elasticsearch Configurations
# Supply your elasticsearch serverless cloud id and api key
ELASTIC_URL = os.getenv('ELASTIC_URL')
ELASTIC_API_KEY = os.getenv('ELASTIC_API_KEY')

##Do not modify
INDEX_NAME = "properties"
TEMPLATE_ID="properties-search-template"
DATA_FILE= "./properties.jsonl"
INFERENCE_ID="e5-endpoint"
MODEL_ID=".multilingual-e5-small_linux-x86_64"

es = Elasticsearch(ELASTIC_URL, api_key=ELASTIC_API_KEY, request_timeout=300)
es.info()


ObjectApiResponse({'name': 'instance-0000000002', 'cluster_name': '3ca433fdd5e04f76bb4c37a4adf40b57', 'cluster_uuid': 'K6dOxDfVSVKeYjKiqGzgSA', 'version': {'number': '9.0.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '112859b85d50de2a7e63f73c8fc70b99eea24291', 'build_date': '2025-04-08T15:13:46.049795831Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

## Create ML inference endpoint
We will create set a number of allocations that supports decent ingest throughput and query latency.

In [5]:
def create_text_embedding_endpoint():
    """
    Creates a new text_embedding endpoint in Elasticsearch with explicit min/max allocations and chunk settings.
    """
    url = f"{ELASTIC_URL}/_inference/text_embedding/{INFERENCE_ID}"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"ApiKey {ELASTIC_API_KEY}"
    }
    payload = {
        "service": "elasticsearch",
        "service_settings": {
            "model_id": MODEL_ID,     
            "num_threads": 1,        
            "num_allocations": 10 
        }
    }

    # Make the POST request
    response = requests.put(url, headers=headers, json=payload)

    # Print the response
    if response.status_code == 200:
        print("Text embedding endpoint created successfully:", response.json())
    else:
        print(f"Error: {response.status_code}, {response.text}")

# Call the function to create the endpoint
create_text_embedding_endpoint()

Text embedding endpoint created successfully: {'inference_id': 'e5-endpoint', 'task_type': 'text_embedding', 'service': 'elasticsearch', 'service_settings': {'num_allocations': 10, 'num_threads': 1, 'model_id': '.multilingual-e5-small_linux-x86_64'}, 'chunking_settings': {'strategy': 'sentence', 'max_chunk_size': 250, 'sentence_overlap': 1}}


## Create Elasticsearch index
Creating an index for the property data.

In [6]:
def create_index():
    mapping = {
      "mappings": {
        "dynamic": "false",
        "properties": {
          "annual-tax": {"type": "integer"},
          "full_html": {"type": "text", "index": False},
          "geo_point": {
            "properties": {
              "lat": {"type": "float"},
              "lon": {"type": "float"}
            }
          },
          "location": {"type": "geo_point"},
          "headings": {"type": "text"},
          "home-price": {"type": "integer"},
          "id": {"type": "keyword"},
          "latitude": {"type": "float"},
          "listing-agent-info": {"type": "text"},
          "longitude": {"type": "float"},
          "maintenance-fee": {"type": "integer"},
          "meta_keywords": {"type": "keyword"},
          "number-of-bathrooms": {"type": "float"},
          "number-of-bedrooms": {"type": "float"},
          "property-description": {"type": "text", "copy_to": ["property-description_semantic"]},
          "property-description_semantic": {
            "type": "semantic_text",
            "inference_id": INFERENCE_ID
          },
          "property-features": {"type": "text", 
                                "copy_to": ["property-features_semantic"], 
                                "fields": {"keyword": {"type": "keyword"}}},
          "property-features_semantic": {
            "type": "semantic_text",
            "inference_id": INFERENCE_ID
          },
          "property-status": {"type": "keyword"},
          "square-footage": {"type": "float"},
          "title": {"type": "text"}
        }
      }
    }
    
    es.indices.create(index=INDEX_NAME, body=mapping)
    print(f"✅ Index '{INDEX_NAME}' created.")

create_index()


✅ Index 'properties' created.


## Search Template

Removes the existing properties-search-template if present and replaces it with an updated version. This ensures the template is always current and correctly structured for search operations.

In [17]:
search_template_content = {
    "script": {
        "lang": "mustache",
        "source": """{
            "_source": false,
            "size": 10,
            "fields": [
                "title",
                "annual-tax",
                "maintenance-fee",
                "number-of-bathrooms",
                "number-of-bedrooms",
                "square-footage",
                "home-price",
                "property-features",
                "property-description"
            ],
            "retriever": {
                "linear": {
                    "filter": {
                        "bool": {
                            "must": [
                                {{#distance}}{
                                    "geo_distance": {
                                        "distance": "{{distance}}",
                                        "location": {
                                            "lat": {{latitude}},
                                            "lon": {{longitude}}
                                        }
                                    }
                                }{{/distance}}
                                {{#bedrooms}}{{#distance}},{{/distance}}{
                                    "range": {
                                        "number-of-bedrooms": {
                                            "gte": {{bedrooms}}
                                        }
                                    }
                                }{{/bedrooms}}
                                {{#bathrooms}}{{#distance}}{{^bedrooms}},{{/bedrooms}}{{/distance}}{{#bedrooms}},{{/bedrooms}}{
                                    "range": {
                                        "number-of-bathrooms": {
                                            "gte": {{bathrooms}}
                                        }
                                    }
                                }{{/bathrooms}}
                                {{#tax}},{
                                    "range": {
                                        "annual-tax": {
                                            "lte": {{tax}}
                                        }
                                    }
                                }{{/tax}}
                                {{#maintenance}},{
                                    "range": {
                                        "maintenance-fee": {
                                            "lte": {{maintenance}}
                                        }
                                    }
                                }{{/maintenance}}
                                {{#square_footage}},{
                                    "range": {
                                        "square-footage": {
                                            "gte": {{square_footage}}
                                        }
                                    }
                                }{{/square_footage}}
                                {{#home_price}},{
                                    "range": {
                                        "home-price": {
                                            "lte": {{home_price}}
                                        }
                                    }
                                }{{/home_price}}
                            ]
                        }
                    },
                    "retrievers": [
                        {
                            "retriever": {
                                "standard": {
                                    "query": {
                                        "semantic": {
                                            "field": "property-description_semantic",
                                            "query": "{{query}}"
                                        }
                                    }
                                }
                            },
                            "weight": 0.3,
                            "normalizer": "minmax"
                        },
                        {
                            "retriever": {
                                "standard": {
                                    "query": {
                                        "semantic": {
                                            "field": "property-features_semantic",
                                            "query": "{{query}}"
                                        }
                                    }
                                }
                            },
                            "weight": 0.3,
                            "normalizer": "minmax"
                        }
                        {{#features}},
                        {
                            "retriever": {
                                "standard": {
                                    "query": {
                                        "terms_set": {
                                            "property-features.keyword": {
                                                "terms": [{{features}}],
                                                "minimum_should_match": 1
                                            }
                                        }
                                    }
                                }
                            },
                            "weight": 0.7,
                            "normalizer": "minmax"
                        }
                        {{/features}}
                    ]
                }
            }
        }"""
    }
}

def create_search_template(
    template_id=TEMPLATE_ID, template_content=search_template_content
):
    """Creates a new search template"""
    try:
        es.put_script(id=template_id, body=template_content)
        print(f"Created search template: {template_id}")
    except Exception as e:
        print(f"Error creating template '{template_id}': {e}")

es.delete_script(id=TEMPLATE_ID)
print(f"Deleted existing search template: {TEMPLATE_ID}")

create_search_template()


Deleted existing search template: properties-search-template
Created search template: properties-search-template


## Ingest property data

In [8]:
MAX_RETRIES = 5
INITIAL_DELAY = 1  
BACKOFF_FACTOR = 2
BATCH_SIZE = 200

def bulk_with_retries(es_client, actions, max_retries=MAX_RETRIES):
    attempt = 0
    delay = INITIAL_DELAY
    while attempt < max_retries:
        try:
            helpers.bulk(es_client, actions)
            return
        except (TransportError, ConnectionError) as e:
            attempt += 1
            if attempt >= max_retries:
                raise e
            print(f"⚠️  Bulk insert failed on attempt {attempt}, retrying in {delay}s... ({type(e).__name__}: {e})")
            time.sleep(delay)
            delay *= BACKOFF_FACTOR

def load_data():
    """
    Loads data from a JSONL file into an Elasticsearch index in batches.
    """
    with open(DATA_FILE, 'r') as file:
        total_lines = sum(1 for _ in file)
        file.seek(0)

        overall_progress = tqdm(total=total_lines, desc=f"Overall Progress - {INDEX_NAME}", unit="records", leave=True)
        batch = []

        for line in file:
            record = json.loads(line.strip())
            batch.append({
                "_index": INDEX_NAME,
                "_source": record
            })
            overall_progress.update(1)

            if len(batch) == BATCH_SIZE:
                bulk_with_retries(es, batch)
                batch = []

        if batch:
            bulk_with_retries(es, batch)

        overall_progress.close()

load_data()

Overall Progress - properties:   0%|          | 0/10000 [00:00<?, ?records/s]

## Teardown
Deletes all data in indexes and the ML inference endpoint

In [4]:
confirmation = input(f"Are you sure you want to delete the index and ml inference endpoint? This cannot be undone")
if confirmation.lower() != 'yes':
    print("Operation canceled.")
else:
    try:
        es.delete_script(id=TEMPLATE_ID)
        print(f"Deleted existing search template: {TEMPLATE_ID}")
    except Exception as e:
        if "not_found" in str(e):
            print(f"Search template '{TEMPLATE_ID}' not found, skipping delete.")
        else:
            print(f"Error deleting template '{TEMPLATE_ID}': {e}")
    if es.indices.exists(index=INDEX_NAME):
        es.indices.delete(index=INDEX_NAME)
        print(f"🗑️ Index '{INDEX_NAME}' deleted.")

    url = f"{ELASTIC_URL}/_inference/text_embedding/{INFERENCE_ID}"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"ApiKey {ELASTIC_API_KEY}"
    }

    # Make the DELETE request
    response = requests.delete(url, headers=headers)

    # Print the response
    if response.status_code == 200:
        print("Text embedding endpoint deleted successfully:", response.json())
    else:
        print(f"Error: {response.status_code}, {response.text}")

Deleted existing search template: properties-search-template
🗑️ Index 'properties' deleted.
Text embedding endpoint deleted successfully: {'acknowledged': True, 'pipelines': [], 'indexes': []}
